## *Rodar notebook toda sexta-feira já que às quintas os atletas têm suas habilidades atualizadas após os treinos e os jogos da semana*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db
from datetime import datetime

In [2]:
# Mantendo o padrão de como o sokker mostra as datas nos dados que disponibiliza
agora= str(datetime.now()).split('.')[0]

print("Data:", agora)

Data: 2021-08-06 08:33:45


In [3]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


Como meu clube não pode ter valores nulos, os dos outros usuários sim (não tenho acesso a alguns dados de seus jogadores), eu insiro uma cláusula if antes de iniciar a inserção dos dados na base de dados para conferir.

In [4]:
# URL da API
url= f'https://sokker.org/xml/players-{club_id}.xml'

# Pegando os dados dos atletas
df_players= pega_dados(url, sessao, search= 'player')

# Inserindo a data de hoje que será importante para acompanharmos suas evoluções
df_players= df_players.assign(date_extracted= agora)

# Soma de valores nulos ou duplicados
nulos= df_players.isnull().sum().sum()
duplicados=  df_players.duplicated().sum()


if (nulos > 0) or (duplicados > 0):
    print('Há valores nulos ou duplicados. Confira os dados e tente novamente.')
    
else:
    print('Não há valores nulos ou duplicados. Inserindo dados na base de dados "dados.sokker.db"...')
    
    db= conecta_ao_db('dados_sokker.db')
    curs= db.cursor()
    
    df_players.to_sql('players', con=db, if_exists='append', index= False) 
    print('Dados extraídos com sucesso!')

Não há valores nulos ou duplicados. Inserindo dados na base de dados "dados.sokker.db"...
Dados extraídos com sucesso!


In [5]:
df = pd.read_sql_query("SELECT * FROM players", db)

In [6]:
df

,ID,date_extracted,name,surname,countryID,age,teamID,wage,height,weight,...,skillStamina,skillTeamwork,skillDiscipline,skillPace,skillTechnique,skillPassing,skillKeeper,skillDefending,skillPlaymaking,skillScoring
0,36519432,2021-07-27 14:44:35,Krastan,Apostolov,54,29,79085,33900.0,177.0,71.5,...,9,1,5,15,3,9,10,5,5,3
1,37968035,2021-07-27 14:44:35,Romildo,Barbadiano,21,29,79085,3900.0,171.0,63.9,...,0,1,1,5,8,3,0,8,1,3
2,37968034,2021-07-27 14:44:35,Giovani,Barriguinha,21,27,79085,2600.0,172.0,76.3,...,3,1,0,6,2,5,1,5,1,4
3,36103236,2021-07-27 14:44:35,Ilario,Bartolozzi,10,29,79085,16900.0,169.0,67.3,...,11,1,9,13,7,8,0,8,9,8
4,37968028,2021-07-27 14:44:35,Cecilio,Bazzarella,21,20,79085,4800.0,175.0,77.4,...,1,2,1,7,1,9,0,5,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,37968041,2021-08-06 08:33:45,Serginho,Siakam,21,28,79085,4300.0,177.0,72.8,...,3,2,1,4,7,8,0,8,0,5
72,37690980,2021-08-06 08:33:45,Joel,Sopó,36,21,79085,5600.0,165.0,69.6,...,5,4,4,9,7,7,0,8,2,4
73,37276405,2021-08-06 08:33:45,Artur,Szymkiewicz,1,23,79085,12500.0,179.0,75.3,...,5,3,7,12,9,8,0,5,8,9
74,37533726,2021-08-06 08:33:45,Tazar,Trbojević,39,21,79085,6700.0,164.0,67.4,...,8,5,5,10,3,6,0,7,2,8


In [7]:
len(df)

76

In [8]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()